In [61]:
using Turing
using LimberJack
using CSV
using NPZ
using FITSIO
using LinearAlgebra
using PythonCall
np = pyimport("numpy")

Python module: <module 'numpy' from '/home/jaime/.julia/environments/v1.7/.CondaPkg/env/lib/python3.10/site-packages/numpy/__init__.py'>

# T-Matrices

In [2]:
#println("My id is ", myid(), " and I have ", Threads.nthreads(), " threads")

fol = "DESY1"
data_set = "wlwl_binned25"
meta = np.load(string("../data/", fol, "/", data_set, "_meta.npz"))
files = npzread(string("../data/", fol, "/", data_set, "_files.npz"))

tracers_names = pyconvert(Vector{String}, meta["tracers"])
pairs = pyconvert(Vector{Vector{String}}, meta["pairs"]);
idx = pyconvert(Vector{Int}, meta["idx"])
data_vector = pyconvert(Vector{Float64}, meta["cls"])
cov_tot = pyconvert(Matrix{Float64}, meta["cov"]);

nz_path = "../data/DESY1/binned_25_nzs/"
zs_k0, nz_k0, cov_k0 = get_nzs(nz_path, "DESwl__0_e")
zs_k1, nz_k1, cov_k1 = get_nzs(nz_path, "DESwl__1_e")
zs_k2, nz_k2, cov_k2 = get_nzs(nz_path, "DESwl__2_e")
zs_k3, nz_k3, cov_k3 = get_nzs(nz_path, "DESwl__3_e")

(Float32[0.08, 0.24, 0.40000004, 0.56, 0.71999997, 0.88, 1.04, 1.2, 1.36, 1.52  …  2.48, 2.6399999, 2.8000002, 2.96, 3.12, 3.28, 3.44, 3.6, 3.76, 3.92], [0.017943619938851237, 0.03833361249391642, 0.02440777766652618, 0.01704057110584659, 0.07209707970095824, 0.5874477589199855, 0.4404475338912065, 0.25267170078051504, 0.07897489762454996, 0.046559044416235526  …  0.003504693784923493, 0.0008005300343577989, 0.0008915212767234572, 0.001654256308613907, 0.00018365304981866675, 6.0278451985871785e-6, 0.00013030516410413156, 0.0, 0.00016407164322363078, 0.0], [8.594352947802566e-5, 0.00015300959903383963, 6.306568771298228e-5, 3.429234614312717e-5, 0.00039670597132587213, 0.015829592966321377, 0.008402431013895935, 0.002185796994687759, 0.00023416931931501573, 0.00011486151885369445  …  4.715220874567531e-6, 1.0481660008115229e-6, 1.174087383634798e-6, 2.1830462776788064e-6, 2.4245196013251086e-7, 1.1969533242531295e-8, 1.729004177613967e-7, 4.177009311760509e-9, 2.1683906553690242e-7, 4.

In [3]:
function model_cov0(nz_k0;
               tracers_names=tracers_names,
               pairs=pairs,
               idx=idx,
               cov_tot=cov_tot, 
               files=files) 

    cosmology = Cosmology()
    cosmology.settings.cosmo_type = Real
    nuisances = Dict("A_IA" => 0,
                     "alpha_IA" => 0,

                     "DESwl__0_e_nz" => nz_k0,
                     "DESwl__1_e_nz" => nz_k1,
                     "DESwl__2_e_nz" => nz_k2,
                     "DESwl__3_e_nz" => nz_k3,
        
                     "DESwl__0_e_m" => 0.012,
                     "DESwl__1_e_m" => 0.012,
                     "DESwl__2_e_m" => 0.012,
                     "DESwl__3_e_m" => 0.012)
    
    return Theory(cosmology, tracers_names, pairs,
                  idx, files; Nuisances=nuisances)
end;

function model_cov1(nz_k1;
               tracers_names=tracers_names,
               pairs=pairs,
               idx=idx,
               cov_tot=cov_tot, 
               files=files) 

    cosmology = Cosmology()
    cosmology.settings.cosmo_type = Real
    nuisances = Dict("A_IA" => 0,
                     "alpha_IA" => 0,

                     "DESwl__0_e_nz" => nz_k0,
                     "DESwl__1_e_nz" => nz_k1,
                     "DESwl__2_e_nz" => nz_k2,
                     "DESwl__3_e_nz" => nz_k3,
        
                     "DESwl__0_e_m" => 0.012,
                     "DESwl__1_e_m" => 0.012,
                     "DESwl__2_e_m" => 0.012,
                     "DESwl__3_e_m" => 0.012)
    
    return Theory(cosmology, tracers_names, pairs,
                  idx, files; Nuisances=nuisances)
end;

function model_cov2(nz_k2;
               tracers_names=tracers_names,
               pairs=pairs,
               idx=idx,
               cov_tot=cov_tot, 
               files=files) 

    cosmology = Cosmology()
    cosmology.settings.cosmo_type = Real
    nuisances = Dict("A_IA" => 0,
                     "alpha_IA" => 0,

                     "DESwl__0_e_nz" => nz_k0,
                     "DESwl__1_e_nz" => nz_k1,
                     "DESwl__2_e_nz" => nz_k2,
                     "DESwl__3_e_nz" => nz_k3,
        
                     "DESwl__0_e_m" => 0.012,
                     "DESwl__1_e_m" => 0.012,
                     "DESwl__2_e_m" => 0.012,
                     "DESwl__3_e_m" => 0.012)
    
    return Theory(cosmology, tracers_names, pairs,
                  idx, files; Nuisances=nuisances)
end;

function model_cov3(nz_k3;
               tracers_names=tracers_names,
               pairs=pairs,
               idx=idx,
               cov_tot=cov_tot, 
               files=files) 

    cosmology = Cosmology()
    cosmology.settings.cosmo_type = Real
    nuisances = Dict("A_IA" => 0,
                     "alpha_IA" => 0,

                     "DESwl__0_e_nz" => nz_k0,
                     "DESwl__1_e_nz" => nz_k1,
                     "DESwl__2_e_nz" => nz_k2,
                     "DESwl__3_e_nz" => nz_k3,
        
                     "DESwl__0_e_m" => 0.012,
                     "DESwl__1_e_m" => 0.012,
                     "DESwl__2_e_m" => 0.012,
                     "DESwl__3_e_m" => 0.012)
    
    return Theory(cosmology, tracers_names, pairs,
                  idx, files; Nuisances=nuisances)
end;

In [4]:
d = 0.0005
T0_num = zeros(240, 25)
T1_num = zeros(240, 25)
T2_num = zeros(240, 25)
T3_num = zeros(240, 25)

for i in 1:25
    e = zeros(25)
    e[i] = d
    T0_num[:, i] = (model_cov0(nz_k0+e) .- model_cov0(nz_k0)) ./ d
    T1_num[:, i] = (model_cov1(nz_k1+e) .- model_cov1(nz_k1)) ./ d
    T2_num[:, i] = (model_cov2(nz_k2+e) .- model_cov2(nz_k2)) ./ d
    T3_num[:, i] = (model_cov3(nz_k3+e) .- model_cov3(nz_k3)) ./ d
end

In [5]:
using ForwardDiff

In [6]:
T0 = ForwardDiff.jacobian(model_cov0, nz_k0);

In [7]:
T1 = ForwardDiff.jacobian(model_cov1, nz_k1);

In [8]:
T2 = ForwardDiff.jacobian(model_cov2, nz_k2);

In [9]:
T3 = ForwardDiff.jacobian(model_cov3, nz_k3);

In [10]:
(T0.-T0_num) ./ T0_num

240×25 Matrix{Float64}:
   0.000193629    0.000395687    0.00025788   …   -0.00200782   -0.00103276
   0.000194802    0.000400604    0.000248758      -0.00291767   -0.00150338
   0.000195359    0.000403243    0.000241545      -0.00370462   -0.00191558
   0.000195694    0.00040488     0.000235744      -0.00434041   -0.00224677
   0.000195919    0.000405966    0.000230903      -0.00487906   -0.00252853
   0.000196085    0.000406835    0.000226822  …   -0.00536115   -0.00278428
   0.000196211    0.000407458    0.000223369      -0.00576375   -0.0029969
   0.000196319    0.000408041    0.000220175      -0.00611734   -0.00318254
   0.000196413    0.00040851     0.000217152      -0.00645072   -0.00335712
   0.000196501    0.000408928    0.000213911      -0.00677781   -0.00352832
   0.000196586    0.000409319    0.000210701  …   -0.00708667   -0.00368951
   0.000196658    0.000409732    0.000207694      -0.00737613   -0.00384022
   0.000196728    0.000410146    0.000204667      -0.00778673   -

In [46]:
TT0 = T0 * Diagonal(cov_k0) * T0'
TT1 = T1 * Diagonal(cov_k1) * T1'
TT2 = T2 * Diagonal(cov_k2) * T2'
TT3 = T3 * Diagonal(cov_k3) * T3';

TT0_num = T0_num * Diagonal(cov_k0) * T0_num'
TT1_num = T1_num * Diagonal(cov_k1) * T1_num'
TT2_num = T2_num * Diagonal(cov_k2) * T2_num'
TT3_num = T3_num * Diagonal(cov_k3) * T3_num';

In [31]:
isposdef(TT0)

false

In [32]:
ishermitian(TT0)

false

In [40]:
issymmetric(TT0)

false

In [50]:
eigen(Hermitian(TT0_num))

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
240-element Vector{Float64}:
 -1.128474576789396e-35
 -7.277266810614447e-36
 -6.206610172341678e-36
 -3.3789720629850065e-36
 -1.3068808330824857e-36
 -2.951794520708943e-37
 -2.5301956620524916e-37
 -2.483149095173731e-37
 -1.65366925868244e-37
 -1.3873920979428907e-37
 -8.33034211261796e-38
 -8.215626234068381e-38
 -4.150222570748722e-38
  ⋮
  6.9982450004057165e-34
  5.786836311304491e-33
  2.1415749709640021e-32
  7.092307165714e-31
  1.0711757829062574e-29
  5.138076635775946e-29
  2.4692232469578864e-27
  1.742463843489843e-26
  7.58311470323733e-25
  3.770160576656751e-23
  3.2355705754491424e-21
  4.198870877301531e-19
vectors:
240×240 Matrix{Float64}:
 -0.00351283   0.018338     -0.00256421  …  0.641099    -0.377669
  0.0795651   -0.102236      0.030915       0.249102    -0.18747
 -0.0669518    0.123657      0.0382648      0.121068    -0.108539
 -0.0110264    0.0157477    -0.0156316      0.0675934   -0.0694481


In [51]:
(Hermitian(TT0_num) - TT0_num) ./ TT0_num

240×240 Matrix{Float64}:
   0.0            0.0          …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            0.0             NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            0.0             NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            0.0             NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            0.0             NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            0.0          …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
   1.826e-16     -1.84292e-16     NaN  NaN  NaN  NaN  NaN  NaN  NaN
  -1.19149e-16   -1.20235e-16     NaN  NaN  NaN  NaN  NaN  NaN  NaN
   1.55571e-16    0.0             NaN  NaN  NaN  NaN  NaN  NaN  NaN
   2.04628e-16   -2.06434e-16     NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0           -1.36052e-16  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            0.0             NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            0.0             NaN  NaN  NaN  NaN  NaN  NaN  NaN
   ⋮                           ⋱              ⋮                 
 NaN            NaN       

In [ ]:
chain = sample(model(fake_data), NUTS(adaptation, TAP, ),
                       MCMCDistributed(), iterations, nchains, progress=true; save_state=true)